In [1]:
import csv
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
import requests
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import datetime as dt
from selenium.webdriver.support.ui import Select

In [8]:
year_start = 2024
year_end = 2024
month_start = 3
month_end = 3
day_start = 1

grab_events = ['60 meter dash', '100 meter', '200 meter', '400 meter', '800 meter', '1600 meter', '3200 meter', 
               '110 meter hurdles', '100 meter hurdles', '300 meter hurdles', 'shot put', 'discus', 'high jump', 
               'long jump', 'triple jump', 'pole vault'] # '4x100 relay', '4x400 relay', '4x800 relay',

grab_events.extend([e + '*' for e in grab_events])

skipped_ids = []
omitted_names = []
headers = ['Placement', 'Grade', 'Athlete', 'blank_1', 'Mark', 'Team', 'blank_2', "Gender", "Event", "Meet", "Year"]


In [10]:
# Launch a browser and website 
driver = webdriver.Chrome()

# pause to allow log in to compete
time.sleep(5)


for year_ in range(year_start, year_end+1):

# --- get event ids for that year --- #
    event_ids = []

    # cycle through months
    for m in range(month_start, month_end+1):
        if m == month_start and year_ == year_start:
            driver.get(f'https://www.athletic.net/events/us/nebraska/{year_}-{m}-{day_start};level=4;sport=9')
        else:
            driver.get(f'https://www.athletic.net/events/us/nebraska/{year_}-{m}-1;level=4;sport=9')
            
            
        time.sleep(5) # time for page to load

        # get page content
        page_source = driver.page_source

        # Parse the page source with BeautifulSoup
        soup = BeautifulSoup(page_source, 'html.parser')


        try:
            select = Select(driver.find_element(By.XPATH, '//*[@id="anetMain"]/anet-site-app/web-events-controller/div/web-find-event/div/div[2]/div[2]/web-events-list/div[27]/div/select'))
            select.select_by_visible_text("All")
            time.sleep(1)

        except: pass


        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        rows = soup.find_all("web-single-event")

        for row in rows:
            event_txt = row.text.strip().replace(" ", "")
            if event_txt[-2:] != "|1":
                event_id = row.get('id', "Event ID Not Found")
                event_ids.append(event_id)


    print(f"{year_} Num Meets: {len(event_ids)}")    
    
    
    
    
# --- create agg_data fo each year --- #
    agg_data = pd.DataFrame()

    i = 1
    omit_events = []

    for meet_id in event_ids:
        data = []

        try:
            url = f"https://www.athletic.net/TrackAndField/meet/{meet_id}/results/all"

            driver.get(url)
            time.sleep(2)

            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')

            title = soup.title.text[:-31].lower()

            tables = soup.find_all('table')


            j = 0

            for table in tables:

                if "Womens Results" in table.parent.parent.text:
                    g = 'Girls'
                elif "Mens Results" in table.parent.parent.text:
                    g = 'Boys'
                else:
                    g = None


                rows = table.find_all('tr')

                for row in rows:
                    row_list = []
                    cells = row.find_all('td')

                    for header, cell in zip(headers, cells):
                        cell_content = cell.text.strip()

                        if header == 'Placement' and '.' in cell_content:
                            cell_content = int(cell_content.replace('.', '').replace(' ', ''))

                        row_list.append(cell_content)


                    if len(row) <= 2 and "--relay team" not in row:
                        curr_event = row.text.replace("I", "").lower().strip()
                        curr_event = curr_event.replace("-", "").replace("finals", "").replace("final", "").replace("prelims", "").replace("semi", "").replace(" results", "").replace('meters', "meter").replace("open", "").replace("junior varsity", "").replace("varsity", "").replace("co-ed", "").replace("coed", "").replace("new", "").replace("jv", "").replace("freshman", "").replace('division', "").replace("frosh/soph", "").replace("high school", "").replace("60m", "60 meter dash").replace('200m', "200 meter").replace('100m', "100 meter").replace('110m', "110 meter").replace('300m', "300 meter").replace("boys", "").replace("girls", "").replace(" 1", "").replace(" x", "").replace('class a', "").replace('class b', "").replace('class c', "").replace('class d', "").strip()

                        if "hurdle" in curr_event:
                            if '100 meter' in curr_event:
                                curr_event = "100 meter hurdles"

                            if '110 meter' in curr_event:
                                curr_event = "110 meter hurdles"

                            if '200 meter' in curr_event:
                                curr_event = "200 meter hurdles"  

                            if '400 meter' in curr_event:
                                curr_event = "400 meter hurdles" 


                        elif "mile" in curr_event:
                            if "1 mile" in curr_event:
                                curr_event = "1600 meter*"

                            elif "2 miles" in curr_event:
                                curr_event = "3200 meter*"                            

                        elif "yard" in curr_event:
                            if "220 yards" in curr_event:
                                curr_event = "200 meter*"                    

                            elif "440 yards" in curr_event:
                                curr_event = "400 meter*"                

                            elif "880 yards" in curr_event:
                                curr_event = "800 meter*"                      

                        if "discus" in curr_event:
                            curr_event = "discus"

                        elif "shot put" in curr_event:
                            curr_event = "shot put"


                    elif curr_event in grab_events:
                        row_list.append(g)
                        row_list.append(curr_event)
                        row_list.append(title)
                        row_list.append(year_)
                        data.append(row_list)


                    else: 
                        if j == 0 and curr_event not in omit_events:
                            print(curr_event)
                            omit_events.append(curr_event)

                        j += 1


            temp_df = pd.DataFrame(data)
            agg_data = pd.concat([agg_data, temp_df], ignore_index=True)

            if i % 10 == 0 or i == len(event_ids) or i == len(event_ids)-1 or i == 1:
                print(f"{i}/{len(event_ids)}")

            i += 1

        except:
            print(f'SKIPPED - {meet_id}')
            skipped_ids.append(meet_id)

    print("Omitted Events:", omit_events)
    print(f"{year_} Done\n{'-'*15}\n")


    # --- format df and write csv --- #
    agg_df = agg_data.copy()
    agg_df.columns = headers

    try:
        agg_df.drop(columns=['blank_1', 'blank_2'], inplace=True)
    except:
        pass


    agg_df[['PR', 'SR']] = 0

    def get_pr_sr(row):
        if "PR" in row['Mark']:
            row['PR'] = 1
            row['SR'] = 1
        elif "SR" in row['Mark']:
            row['SR'] = 1
        return row

    master_df = agg_df.apply(get_pr_sr, axis=1)       

    master_df['Mark'] = master_df['Mark'].apply(lambda x: x.replace('PR', '').replace('SR', ''))

    master_df = master_df[['Placement', 'Grade', 'Athlete', 'Mark', 'PR', 'SR', 'Team', "Gender", 'Event', 'Meet', "Year"]]

    master_df[(master_df['Event'] == '4x400 relay') | (master_df['Event'].isna())]

    grade_dict = {"FR":"9", "SO":"10", "Jr":"11", "JR":"11", "SR":"12", "09":"9"}

    master_df['Grade'] = master_df['Grade'].apply(lambda x: grade_dict[x] if x in grade_dict.keys() else x)
    master_df['Mark'] = master_df['Mark'].apply(lambda x: x.replace('a', '').replace('m', '')) #['Team'].value_counts()


    if m == month_start and year_== year_start:
        master_df.to_csv(f"{year_}_TF_data_through_{m}.csv", index=False)
        
    else:
        master_df.to_csv(f"{year_}_TF_data.csv", index=False)


driver.quit()

2024 Num Meets: 92
60 meter dash hurdles
1/92
55 meter dash
10/92
4x100 relay
60 meter dash unified
20/92
4x400 relay
30/92
40/92
100 meter unified
50/92
100 meter mixed
60 yard dash
60/92
70/92
100 meter  unified unified
80/92
90/92
91/92
92/92
Omitted Events: ['60 meter dash hurdles', '55 meter dash', '4x100 relay', '60 meter dash unified', '4x400 relay', '100 meter unified', '100 meter mixed', '60 yard dash', '100 meter  unified unified']
2024 Done
---------------



NameError: name 'start_day' is not defined

In [6]:
agg_data

,0,1,2,3,4,5,6,7,8,9,10
0,1,10,Jonathan Linares,,2:16.59aPR,Fremont,,Boys,800 meter,dc west dual,2024
1,2,10,Michael Nichols,,2:17.94aPR,Fremont,,Boys,800 meter,dc west dual,2024
2,3,12,Darby Giles,,2:19.32aPR,Douglas County West,,Boys,800 meter,dc west dual,2024
3,4,9,Aden Soto,,2:20.08aPR,Fremont,,Boys,800 meter,dc west dual,2024
4,5,10,Jaxon Pleskac,,2:20.89aPR,Fremont,,Boys,800 meter,dc west dual,2024
...,...,...,...,...,...,...,...,...,...,...,...
35826,18,11,Evonna Bahwawsi,,28-09.25SR,Omaha Central,,Girls,triple jump,westside invitational,2024
35827,19,9,Sophia Markle,,28-05.50,Omaha Westside,,Girls,triple jump,westside invitational,2024
35828,19,10,Jadyn Bedel,,28-05.50,Millard West,,Girls,triple jump,westside invitational,2024
35829,21,9,Olivia Bader,,27-10.50,Millard West,,Girls,triple jump,westside invitational,2024


In [ ]:
# page_source = driver.page_source
# soup = BeautifulSoup(page_source, 'html.parser')


# try:
#     event_ids
    
# # except:
# event_ids = []

# # cycle through months
# for m in range(month_start, (month_end+1)):
#     driver.get(f'https://www.athletic.net/events/us/nebraska/{year}-{m}-1;level=4;sport=9')
#     time.sleep(5) # time for page to load

#     # get page content
#     page_source = driver.page_source

#     # Parse the page source with BeautifulSoup
#     soup = BeautifulSoup(page_source, 'html.parser')


#     try:
#         select = Select(driver.find_element(By.XPATH, '//*[@id="anetMain"]/anet-site-app/web-events-controller/div/web-find-event/div/div[2]/div[2]/web-events-list/div[27]/div/select'))
#         select.select_by_visible_text("All")
#         time.sleep(1)

#     except: pass


#     page_source = driver.page_source
#     soup = BeautifulSoup(page_source, 'html.parser')

#     rows = soup.find_all("web-single-event")

#     for row in rows:
#         event_txt = row.text.strip().replace(" ", "")
#         if event_txt[-2:] != "|1":
#             event_id = row.get('id', "Event ID Not Found")
#             event_ids.append(event_id)


# print(len(event_ids))
# # print(event_ids)


In [ ]:
# # --- create agg_data fo each year --- #
# agg_data = pd.DataFrame()

# i = 1
# omit_events = []

# for meet_id in event_ids:
#     data = []

#     try:
#         url = f"https://www.athletic.net/TrackAndField/meet/{meet_id}/results/all"

#         driver.get(url)
#         time.sleep(2)

#         page_source = driver.page_source
#         soup = BeautifulSoup(page_source, 'html.parser')

#         title = soup.title.text[:-31].lower()

#         tables = soup.find_all('table')


#         j = 0

#         for table in tables:

#             if "Womens Results" in table.parent.parent.text:
#                 g = 'Girls'
#             elif "Mens Results" in table.parent.parent.text:
#                 g = 'Boys'
#             else:
#                 g = None


#             rows = table.find_all('tr')

#             for row in rows:
#                 row_list = []
#                 cells = row.find_all('td')

#                 for header, cell in zip(headers, cells):
#                     cell_content = cell.text.strip()

#                     if header == 'Placement' and '.' in cell_content:
#                         cell_content = int(cell_content.replace('.', '').replace(' ', ''))

#                     row_list.append(cell_content)


#                 if len(row) <= 2 and "--relay team" not in row:
#                     curr_event = row.text.replace("I", "").lower().strip()
#                     curr_event = curr_event.replace("-", "").replace("finals", "").replace("final", "").replace("prelims", "").replace("semi", "").replace(" results", "").replace('meters', "meter").replace("open", "").replace("junior varsity", "").replace("varsity", "").replace("co-ed", "").replace("coed", "").replace("new", "").replace("jv", "").replace("freshman", "").replace('division', "").replace("frosh/soph", "").replace("high school", "").replace("60m", "60 meter dash").replace('200m', "200 meter").replace('100m', "100 meter").replace('110m', "110 meter").replace('300m', "300 meter").replace("boys", "").replace("girls", "").replace(" 1", "").replace(" x", "").replace('class a', "").replace('class b', "").replace('class c', "").replace('class d', "").strip()

#                     if "hurdle" in curr_event:
#                         if '100 meter' in curr_event:
#                             curr_event = "100 meter hurdles"

#                         if '110 meter' in curr_event:
#                             curr_event = "110 meter hurdles"

#                         if '200 meter' in curr_event:
#                             curr_event = "200 meter hurdles"  

#                         if '400 meter' in curr_event:
#                             curr_event = "400 meter hurdles" 


#                     elif "mile" in curr_event:
#                         if "1 mile" in curr_event:
#                             curr_event = "1600 meter*"

#                         elif "2 miles" in curr_event:
#                             curr_event = "3200 meter*"                            

#                     elif "yard" in curr_event:
#                         if "220 yards" in curr_event:
#                             curr_event = "200 meter*"                    

#                         elif "440 yards" in curr_event:
#                             curr_event = "400 meter*"                

#                         elif "880 yards" in curr_event:
#                             curr_event = "800 meter*"                      

#                     if "discus" in curr_event:
#                         curr_event = "discus"

#                     elif "shot put" in curr_event:
#                         curr_event = "shot put"


#                 elif curr_event in grab_events:
#                     row_list.append(g)
#                     row_list.append(curr_event)
#                     row_list.append(title)
#                     row_list.append(year_)
#                     data.append(row_list)


#                 else: 
#                     if j == 0 and curr_event not in omit_events:
#                         print(curr_event)
#                         omit_events.append(curr_event)

#                     j += 1


#         temp_df = pd.DataFrame(data)
#         agg_data = pd.concat([agg_data, temp_df], ignore_index=True)

#         if i % 10 == 0 or i == len(event_ids) or i == len(event_ids)-1 or i == 1:
#             print(f"{i}/{len(event_ids)}")

#         i += 1

#     except:
#         print(f'SKIPPED - {meet_id}')
#         skipped_ids.append(meet_id)

# print("Omitted Events:", omit_events)



In [ ]:
# agg_df = agg_data.copy()
# agg_df.columns = headers

# try:
#     agg_df.drop(columns=['blank_1', 'blank_2'], inplace=True)
# except:
#     pass


# agg_df[['PR', 'SR']] = 0

# def get_pr_sr(row):
#     if "PR" in row['Mark']:
#         row['PR'] = 1
#         row['SR'] = 1
#     elif "SR" in row['Mark']:
#         row['SR'] = 1
#     return row
        
# master_df = agg_df.apply(get_pr_sr, axis=1)       

# master_df['Mark'] = master_df['Mark'].apply(lambda x: x.replace('PR', '').replace('SR', ''))

# master_df = master_df[['Placement', 'Grade', 'Athlete', 'Mark', 'PR', 'SR', 'Team', 'Event', 'Meet', "Year"]]

# master_df[(master_df['Event'] == '4x400 relay') | (master_df['Event'].isna())]

# grade_dict = {"FR":"9", "SO":"10", "Jr":"11", "JR":"11", "SR":"12", "09":"9"}

# master_df['Grade'] = master_df['Grade'].apply(lambda x: grade_dict[x] if x in grade_dict.keys() else x)
# master_df['Mark'] = master_df['Mark'].apply(lambda x: x.replace('a', '').replace('m', '')) #['Team'].value_counts()


# import datetime as dt
# curr_date = dt.date.today().strftime('$Y_%m_%d')
# master_df.to_csv(f"{curr_date}_TF_data.csv", index=False)


In [ ]:
# # 4x100 Throwers Relay
# # 60y Hurdles

    #                 if header == 'Event':
    #                     if "relay" in curr_event:
    #                         relay_parts = []
    #                         athlete_list = []
    #                         row_div = row.text.split()

    #                         if '--' not in row_div[0]:
    #                             # Split the row content for relay events
    #                             relay_parts.append(row_div[0].replace(" ", "").replace(".", "")) # placement
    #                             relay_parts.append('--') # no grade for relays

    #                             # list of athletes, their grade, and split
    #                             athlete_list.append(" ".join(row_div[1:4]))
    #                             athlete_list.append(" ".join(row_div[4:7]))
    #                             athlete_list.append(" ".join(row_div[7:10]))
    #                             athlete_list.append(" ".join(row_div[10:13]))
    #                             relay_parts.append(athlete_list)

    #                             relay_parts.append('') # blank_1
    #                             relay_parts.append(row_div[13]) # Mark
    #                             relay_parts.append(" ".join(row_div[14:])) # Team
    #                             relay_parts.append('') # blank_2          

    #                             row_list = relay_parts

    # #                         elif row_div



    #                         else: 
    #                             relay_parts.append('--')
    #                             relay_parts.append('--') # no grade for relays

    #                             # athletes
    #                             relay_parts.append(row_div[:2])

    #                             relay_parts.append('') # blank_1
    #                             relay_parts.append(row_div[2]) # Mark
    #                             relay_parts.append(" ".join(row_div[3:])) # Team
    #                             relay_parts.append('') # blank_2                            

    #                             row_list = relay_parts

    # #                         print(row_list)


    #                     else: 
    #                         row_list.append(cell_content)

    #                 else:
    #                     row_list.append(cell_content)
# # 4x440 Yard Relay
# # 4x880 Yard Relay
    